In [11]:
import requests
import json

Config Meta App + Thread Variables

In [13]:
import json

with open('serraoshawn9_config.json', 'r') as file:
    config = json.load(file)

app_id = config["app_id"]
app_secret = config["app_secret"]

redirect_uri = "https://287095596fb5.ngrok-free.app"


Get User Access Token

In [14]:
url = "https://threads.net/oauth/authorize?"
url = url + f"client_id={int(app_id)}"
url = url + "&" + f"redirect_uri={redirect_uri}"
url = url + "&" + f"scope={('threads_basic,threads_content_publish,threads_read_replies,threads_manage_replies,threads_manage_insights').replace(',', '%2C')}"
url = url + "&" + f"response_type=code"
url


'https://threads.net/oauth/authorize?client_id=781251690972294&redirect_uri=https://287095596fb5.ngrok-free.app&scope=threads_basic%2Cthreads_content_publish%2Cthreads_read_replies%2Cthreads_manage_replies%2Cthreads_manage_insights&response_type=code'

paste   url from browser and get authentication code

In [15]:
from urllib.parse import urlparse, parse_qs

returned_url = "https://287095596fb5.ngrok-free.app/?code=AQBHBjRezBt2y3-wxdnz6MeHfBGHu_pTDt9uDRlnbSHqLGIDuPb6h2gi_J7Lk3Hv0SEKPMASoj-hbzy9QWo51IzykamtJ6r_UGz-KWQ_XSrqY5OwGVPNZWr96BmDpggwWANmNLswPw1xiB3HySCMVmSVVaEb5ccS9_56iRbBEUuLMhDasN1fT_a0AIxHlCQfePnBYJkX-6_de8Zund_Ew5AOz03_leTw6J6fTGC8Bi8YEA#_"

parsed = urlparse(returned_url)
query_params = parse_qs(parsed.query)

authorization_code = query_params["code"][0]

authorization_code


'AQBHBjRezBt2y3-wxdnz6MeHfBGHu_pTDt9uDRlnbSHqLGIDuPb6h2gi_J7Lk3Hv0SEKPMASoj-hbzy9QWo51IzykamtJ6r_UGz-KWQ_XSrqY5OwGVPNZWr96BmDpggwWANmNLswPw1xiB3HySCMVmSVVaEb5ccS9_56iRbBEUuLMhDasN1fT_a0AIxHlCQfePnBYJkX-6_de8Zund_Ew5AOz03_leTw6J6fTGC8Bi8YEA'

Access Token

In [16]:
url = "https://graph.threads.net/oauth/access_token"

form_data = {
    "client_id": int(app_id),
    "client_secret": app_secret,
    "grant_type": "authorization_code",
    "redirect_uri": redirect_uri,
    "code": authorization_code
}

response = requests.post(url, data=form_data)
data = response.json()

# Always print full response
print(json.dumps(data, indent=4))

# Safe extraction (no crash)
if "access_token" in data:
    user_access_token = data["access_token"]
    print("\n✅ Your user token is:")
    print(user_access_token)
else:
    print("\n❌ Access token not returned by API")
    print("OAuth error — see response above")


{
    "access_token": "THAALGi1WFRIZABUVE1czFnd3dUYzg2QXJpNzcxR3RxR3BPRE80WkVYS1M3QzJScDZAXMzgzckJXNk1rWHpOSm03VGM5TGJ0MVFfNDZAqbUtVMGh0QnJJRUJ0YjliMmYxQmlfMTFoUVNwcVVOSkdJZAkFzTXpVYnBaWXZAqejByek84bUgyTk1MT3VqYklSQjFuaC1FUXVCeEV3NGJZAUG54ZAk5iWk1KMFUZD",
    "user_id": 25591090347251858
}

✅ Your user token is:
THAALGi1WFRIZABUVE1czFnd3dUYzg2QXJpNzcxR3RxR3BPRE80WkVYS1M3QzJScDZAXMzgzckJXNk1rWHpOSm03VGM5TGJ0MVFfNDZAqbUtVMGh0QnJJRUJ0YjliMmYxQmlfMTFoUVNwcVVOSkdJZAkFzTXpVYnBaWXZAqejByek84bUgyTk1MT3VqYklSQjFuaC1FUXVCeEV3NGJZAUG54ZAk5iWk1KMFUZD


Get Long Access token

In [17]:
url = f"https://graph.threads.net/access_token"

payload = {
    "grant_type": "th_exchange_token",
    "client_secret": app_secret,
    "access_token": user_access_token
}

response = requests.get(url, params=payload)
data = response.json()

long_access_token = data["access_token"]

print("Your long token is : " + long_access_token)


Your long token is : THAALGi1WFRIZABUVNlSzdfd3JiZA2dvODdmOGhUT2ZApMDlnNXF5X2dKVmR1OXpOR1BVM1pCVm9HZAzJSVzVYeHRKME9rSnlWMWxpSFFOYnEtbWNwU0NFYmhhTUJmOFFQQy04MjlLWXo5OExfRnU0NWs0bjl1QUlqVVZAndjdxSHZAuNkc1ZAwZDZD


Test API

In [18]:
url = "https://graph.threads.net/v1.0/me"

payload = {
    "fields": "id,username,name,threads_profile_picture_url,threads_biography,is_eligible_for_geo_gating",
    "access_token": long_access_token
}

response = requests.get(url, params=payload)
data = response.json()

print(json.dumps(data, indent=4))
print(f"Thread User ID : {data['id']}")


{
    "id": "25591090347251858",
    "username": "serraoshawn9",
    "name": "Shawn Serrao",
    "is_eligible_for_geo_gating": false
}
Thread User ID : 25591090347251858


Get User Theads

In [22]:
# Get user profile first
profile_url = "https://graph.threads.net/v1.0/me"

profile_payload = {
    "fields": "id,username",
    "access_token": long_access_token
}

profile_response = requests.get(profile_url, params=profile_payload)
profile_data = profile_response.json()

mine_thread_id = profile_data["id"]

print("Thread User ID:", mine_thread_id)


Thread User ID: 25591090347251858


In [1]:
import requests
import json

# ==============================
# LONG ACCESS TOKEN (string!)
# ==============================
long_access_token = "THAALGi1WFRIZABUVNlSzdfd3JiZA2dvODdmOGhUT2ZApMDlnNXF5X2dKVmR1OXpOR1BVM1pCVm9HZAzJSVzVYeHRKME9rSnlWMWxpSFFOYnEtbWNwU0NFYmhhTUJmOFFQQy04MjlLWXo5OExfRnU0NWs0bjl1QUlqVVZAndjdxSHZAuNkc1ZAwZDZD"

BASE_URL = "https://graph.threads.net/v1.0"

# ==============================
# STEP 1 — Get your Threads user ID
# ==============================
profile_url = f"{BASE_URL}/me"
profile_payload = {
    "fields": "id,username",
    "access_token": long_access_token
}

profile_response = requests.get(profile_url, params=profile_payload)
profile_data = profile_response.json()

if "error" in profile_data:
    print("ERROR:", profile_data)
    exit()

mine_thread_id = profile_data["id"]
print("Thread User ID:", mine_thread_id)

# ==============================
# STEP 2 — Fetch Threads (with pagination)
# ==============================
url = f"{BASE_URL}/{mine_thread_id}/threads"

payload = {
    "fields": "id,media_product_type,media_type,media_url,permalink,owner,username,text,timestamp,shortcode,thumbnail_url,children,is_quote_post,quoted_post",
    "limit": 25,
    "access_token": long_access_token
}

all_threads = []

while True:
    response = requests.get(url, params=payload)
    data = response.json()

    if "error" in data:
        print("ERROR:", data)
        break

    all_threads.extend(data.get("data", []))

    # Pagination
    paging = data.get("paging", {})
    next_url = paging.get("next")

    if not next_url:
        break

    url = next_url
    payload = {}  # next_url already contains params

# ==============================
# OUTPUT
# ==============================
print(json.dumps(all_threads, indent=4))
print(f"\nTotal Threads Fetched: {len(all_threads)}")


Thread User ID: 25591090347251858
[
    {
        "id": "18088866674512783",
        "media_product_type": "THREADS",
        "media_type": "TEXT_POST",
        "permalink": "https://www.threads.com/@serraoshawn9/post/DUK39hxkVPO",
        "owner": {
            "id": "25591090347251858"
        },
        "username": "serraoshawn9",
        "text": "New thread \u2705",
        "timestamp": "2026-01-31T09:23:59+0000",
        "shortcode": "DUK39hxkVPO",
        "is_quote_post": false
    }
]

Total Threads Fetched: 1


Get User Replies

In [2]:
import requests
import json

BASE_URL = "https://graph.threads.net/v1.0"

# -----------------------------
# Step 1: Get your profile ID
# -----------------------------
profile_url = f"{BASE_URL}/me"
profile_payload = {
    "fields": "id,username",
    "access_token": long_access_token
}

profile_data = requests.get(profile_url, params=profile_payload).json()
my_user_id = profile_data["id"]

print("My Threads ID:", my_user_id)

# -----------------------------
# Step 2: Get YOUR replies
# -----------------------------
replies_url = f"{BASE_URL}/{my_user_id}/replies"

payload = {
    "fields": "id,text,username,permalink,timestamp,media_product_type,media_type,media_url,shortcode,thumbnail_url,is_quote_post,has_replies",
    "limit": 50,
    "access_token": long_access_token
}

all_replies = []

while True:
    r = requests.get(replies_url, params=payload).json()

    if "error" in r:
        print("ERROR:", r)
        break

    all_replies.extend(r.get("data", []))

    paging = r.get("paging", {})
    next_url = paging.get("next")

    if not next_url:
        break

    replies_url = next_url
    payload = {}

# -----------------------------
# Output
# -----------------------------
print(json.dumps(all_replies, indent=4))
print(f"\nTotal Self Replies Found: {len(all_replies)}")

# -----------------------------
# Save
# -----------------------------
with open("my_replies.json", "w") as f:
    json.dump(all_replies, f, indent=4)

print("✅ Saved to my_replies.json")


My Threads ID: 25591090347251858
[
    {
        "id": "17861460411416410",
        "text": "Terrible post, wasted my time.\ud83e\udd2c",
        "username": "serraoshawn9",
        "permalink": "https://www.threads.com/@serraoshawn9/post/DOTpJmxkt4Q",
        "timestamp": "2025-09-07T15:59:21+0000",
        "media_product_type": "THREADS",
        "media_type": "TEXT_POST",
        "shortcode": "DOTpJmxkt4Q",
        "is_quote_post": false,
        "has_replies": false
    },
    {
        "id": "18056515106576185",
        "text": "Just average content, nothing much.",
        "username": "serraoshawn9",
        "permalink": "https://www.threads.com/@serraoshawn9/post/DOTpBPKEizC",
        "timestamp": "2025-09-07T15:58:13+0000",
        "media_product_type": "THREADS",
        "media_type": "TEXT_POST",
        "shortcode": "DOTpBPKEizC",
        "is_quote_post": false,
        "has_replies": false
    },
    {
        "id": "18020527496754303",
        "text": "This makes no sense 

Get Public Replies to a Thread using Thread id

In [5]:
thread_media_id = 18088866674512783

url = f"https://graph.threads.net/v1.0/{thread_media_id}/replies"

payload = {
    "fields": "id,text,username,permalink,timestamp,media_product_type,media_type,media_url,shortcode,thumbnail_url,children,is_quote_post,has_replies",
    "reverse": "true",
    "access_token": long_access_token
}

response = requests.get(url, params=payload)
data = response.json()

print(json.dumps(data, indent=4))


{
    "data": [
        {
            "id": "18092880139802115",
            "text": "Reply test \u2705",
            "username": "eron11877",
            "permalink": "https://www.threads.com/@eron11877/post/DUK4BBwgVNn",
            "timestamp": "2026-01-31T09:24:27+0000",
            "media_product_type": "THREADS",
            "media_type": "TEXT_POST",
            "shortcode": "DUK4BBwgVNn",
            "is_quote_post": false,
            "has_replies": false
        }
    ],
    "paging": {
        "cursors": {
            "before": "QVFIU01IU1o0TTFHZAlJzOWRYeU1YSm5idGpnMnh0QTdoa0tBX2NPNW1pSE5QWlBycEJHekJBbkx1SG9ZAdldDVkJDY09XSjU2akJIUW1rbmZACaHBibkdJWFZAn",
            "after": "QVFIU01IU1o0TTFHZAlJzOWRYeU1YSm5idGpnMnh0QTdoa0tBX2NPNW1pSE5QWlBycEJHekJBbkx1SG9ZAdldDVkJDY09XSjU2akJIUW1rbmZACaHBibkdJWFZAn"
        }
    }
}
